# Proyecto

## Parte 1: Entrenamiento y validación y selección

In [31]:
#Importando librerías

import sklearn.metrics as mt
from sklearn.externals import joblib
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
from datetime import datetime
from sklearn import tree
# !pip install graphviz
import graphviz
from scipy import stats

In [32]:
#Habilitando compatibilidad con tensorflow v1
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


## Cargando y examinando datos

In [3]:
data = pd.read_csv("data_titanic_proyecto.csv")
print(data.shape)
print(data.dtypes)
data.head()

(891, 12)
PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Cabin                  object
Embarked               object
passenger_class        object
passenger_sex          object
passenger_survived     object
dtype: object


,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [4]:
#Verificando valos NaN en los datos
pd.DataFrame({'Cantidad de filas NaN': data.isnull().sum()})

,Cantidad de filas NaN
PassengerId,0
Name,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0
Cabin,687
Embarked,2
passenger_class,0


In [5]:
#Reemplazando los valos NaN de Embarked por la moda
embarkedDF=pd.DataFrame(data["Embarked"].value_counts())
data["Embarked"] = data["Embarked"].fillna(embarkedDF['Embarked'].idxmax())

#Descartando la columna Cabin ya que faltan muchos datos
data = data.drop(['Cabin'], axis=1)

#Reemplazando los valos NaN de la edad por la media
meanAge = np.mean(data.Age)
data["Age"] = data["Age"].fillna(meanAge);
pd.DataFrame({'Cantidad de filas NaN': data.isnull().sum()})

,Cantidad de filas NaN
PassengerId,0
Name,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0
Embarked,0
passenger_class,0
passenger_sex,0


In [6]:
#Verificando variables categoricas
print(data["Embarked"].value_counts())
print(data["passenger_class"].value_counts())
print(data["passenger_sex"].value_counts())
print(data["passenger_survived"].value_counts())


S    646
C    168
Q     77
Name: Embarked, dtype: int64
Lower     491
Upper     216
Middle    184
Name: passenger_class, dtype: int64
M    577
F    314
Name: passenger_sex, dtype: int64
N    549
Y    342
Name: passenger_survived, dtype: int64


In [7]:
#Convirtiendo variables categoricas binarias a numericas
vc_binarias = np.array(['passenger_sex','passenger_survived'])
data[['male','survived']] = data.loc[:,vc_binarias].apply(LabelEncoder().fit_transform)  
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Embarked,passenger_class,passenger_sex,passenger_survived,male,survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,S,Lower,M,N,1,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C,Upper,F,Y,0,1
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,S,Lower,F,Y,0,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,S,Upper,F,Y,0,1
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,S,Lower,M,N,1,0


In [8]:
#Convirtiendo variables categoricas no binarias a numericas
vc_multiples = np.array(['Embarked','passenger_class'])
df_vc_multiples = pd.get_dummies(data.loc[:,vc_multiples])
df_vc_multiples.head()
data= pd.concat([data,df_vc_multiples],axis=1,sort=True)
data.head()
data.describe()

,PassengerId,Age,SibSp,Parch,Fare,male,survived,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,29.699118,0.523008,0.381594,32.204208,0.647587,0.383838,0.188552,0.086420,0.725028,0.551066,0.206510,0.242424
std,257.353842,13.002015,1.102743,0.806057,49.693429,0.477990,0.486592,0.391372,0.281141,0.446751,0.497665,0.405028,0.428790
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,29.699118,0.000000,0.000000,14.454200,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
75%,668.500000,35.000000,1.000000,0.000000,31.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,891.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
features = ["Age", "SibSp", "Parch", "Fare", "male","Embarked_C", "Embarked_Q", "Embarked_S",
             "passenger_class_Lower", "passenger_class_Middle", "passenger_class_Upper"]

vectorX= data[features].values
vectorY=data.survived.values


### Separación de los datos en entrenamiento, validación y pruebas

In [10]:
trainX,testX,trainY,testY = train_test_split(vectorX, vectorY,test_size = 0.2,shuffle = True,random_state = 20)
trainX,valX,trainY,valY = train_test_split(trainX, trainY,test_size = 0.15,shuffle = True,random_state = 2020)
np.savez(file = "testDataSet.npz", testX=testX, testY=testY)
trainX.shape, trainY.shape, testX.shape, testY.shape, valX.shape, valY.shape

((605, 11), (605,), (179, 11), (179,), (107, 11), (107,))

### Bitacora de experimentos

In [11]:
def bitacora(ArchivoBitacora, df_bitacora):
    if not os.path.isfile(ArchivoBitacora):
        df_bitacora.to_csv(ArchivoBitacora)
    else: 
        df_bitacora.to_csv(ArchivoBitacora, header=False, mode='a')

## SVM

In [12]:
def train_SVM (kernel, c, gamma, xTrain, yTrain, xVal, yVal, Standardize):
    if (Standardize==True):
        scaler1 = StandardScaler()
        xTrain = scaler1.fit_transform(xTrain)
        scaler2 = StandardScaler()
        xVal = scaler2.fit_transform(xVal)
    modelSVM = svm.SVC(kernel=kernel, C=c, gamma=gamma)
    modelSVM.fit(xTrain, yTrain)
    yHat = modelSVM.predict(xVal)    
    precision = metrics.precision_score(yVal, yHat)
    accuracy = metrics.accuracy_score(yVal, yHat)
    recall = metrics.recall_score(yVal, yHat)
    f1_score = metrics.f1_score(yVal, yHat)
    score = modelSVM.score(xVal, yVal)  
    resultados = { 'kernel':[kernel],'c':[c],'gamma':[gamma],
                   'precision':[precision],'accuracy':[accuracy],
                   'recall':[recall],'f1_score':[f1_score],
                   'score':[score],'Standardize':[Standardize]
                 }
    df_resultados = pd.DataFrame(resultados)
    bitacora('bitacoraSVM.csv',df_resultados)
    return df_resultados,modelSVM

In [13]:
lista_C = np.arange(0.1,1,0.1)
lista_gamma = np.arange(0.01,1,0.01)
# lista_kernel = ['rbf','linear','poly','sigmoid']
resultado_modelos_svm = pd.DataFrame()
modelos_svm = list()

for c in lista_C:
    for gamma in lista_gamma:
        resultados,model_svm = train_SVM('rbf', c, gamma ,trainX, trainY, valX, valY, True)
        resultado_modelos_svm = pd.concat([resultado_modelos_svm,resultados],axis=0,ignore_index=True)
        modelos_svm.append(model_svm)
print(resultado_modelos_svm)
index_best_model = resultado_modelos_svm['score'].idxmax()
joblib.dump(modelos_svm[index_best_model], 'modeloSVM.pkl') 
print(resultado_modelos_svm.loc[index_best_model])

    kernel    c  gamma  precision  accuracy    recall  f1_score     score  \
0      rbf  0.1   0.01   0.916667  0.663551  0.239130  0.379310  0.663551   
1      rbf  0.1   0.02   0.875000  0.738318  0.456522  0.600000  0.738318   
2      rbf  0.1   0.03   0.838710  0.766355  0.565217  0.675325  0.766355   
3      rbf  0.1   0.04   0.838710  0.766355  0.565217  0.675325  0.766355   
4      rbf  0.1   0.05   0.846154  0.738318  0.478261  0.611111  0.738318   
..     ...  ...    ...        ...       ...       ...       ...       ...   
886    rbf  0.9   0.95   0.750000  0.738318  0.586957  0.658537  0.738318   
887    rbf  0.9   0.96   0.750000  0.738318  0.586957  0.658537  0.738318   
888    rbf  0.9   0.97   0.742857  0.728972  0.565217  0.641975  0.728972   
889    rbf  0.9   0.98   0.742857  0.728972  0.565217  0.641975  0.728972   
890    rbf  0.9   0.99   0.742857  0.728972  0.565217  0.641975  0.728972   

     Standardize  
0           True  
1           True  
2           True  

## DecisionTree

In [14]:
def train_DecisionTree(depth,xTrain, yTrain, xVal, yVal, Standardize):
    if (Standardize==True):
        scaler1 = StandardScaler()
        xTrain = scaler1.fit_transform(xTrain)
        scaler2 = StandardScaler()
        xVal = scaler2.fit_transform(xVal)
    modelDecisionTree = tree.DecisionTreeClassifier(max_depth = depth)
    modelDecisionTree.fit(xTrain, yTrain)
    yHat = modelDecisionTree.predict(xVal)    
    precision = metrics.precision_score(yVal, yHat)
    accuracy = metrics.accuracy_score(yVal, yHat)
    recall = metrics.recall_score(yVal, yHat)
    f1_score = metrics.f1_score(yVal, yHat)
    score = modelDecisionTree.score(xVal, yVal)  
    resultados = { 'depth':[depth],'precision':[precision],'accuracy':[accuracy],
                   'recall':[recall],'f1_score':[f1_score],
                   'score':[score],'Standardize':[Standardize]
                 }
    df_resultados = pd.DataFrame(resultados)
    bitacora('bitacoraDecisionTree.csv',df_resultados)
    return df_resultados,modelDecisionTree

In [15]:
lista_depth = np.arange(1,10,1)
resultado_modelos_DecisionTree = pd.DataFrame()
modelos_DecisionTree = list()

for depth in lista_depth:
    resultados,model_DecisionTree = train_DecisionTree(depth,trainX, trainY, valX, valY, True)
    resultado_modelos_DecisionTree = pd.concat([resultado_modelos_DecisionTree,resultados],axis=0,ignore_index=True)
    modelos_DecisionTree.append(model_DecisionTree)
print(resultado_modelos_DecisionTree)
index_best_model = resultado_modelos_DecisionTree['score'].idxmax()
joblib.dump(modelos_DecisionTree[index_best_model], 'modeloDecisionTree.pkl') 
print(resultado_modelos_DecisionTree.loc[index_best_model])

   depth  precision  accuracy    recall  f1_score     score  Standardize
0      1   0.815789  0.794393  0.673913  0.738095  0.794393         True
1      2   0.952381  0.747664  0.434783  0.597015  0.747664         True
2      3   0.878788  0.803738  0.630435  0.734177  0.803738         True
3      4   0.829268  0.822430  0.739130  0.781609  0.822430         True
4      5   0.829268  0.822430  0.739130  0.781609  0.822430         True
5      6   0.755102  0.803738  0.804348  0.778947  0.803738         True
6      7   0.822222  0.841121  0.804348  0.813187  0.841121         True
7      8   0.698113  0.766355  0.804348  0.747475  0.766355         True
8      9   0.734694  0.785047  0.782609  0.757895  0.785047         True
depth                 7
precision      0.822222
accuracy       0.841121
recall         0.804348
f1_score       0.813187
score          0.841121
Standardize        True
Name: 6, dtype: object


### Generando arbol con Graphviz

In [16]:
tree_data = tree.export_graphviz(modelos_DecisionTree[index_best_model],  
                                class_names=data["passenger_survived"].value_counts().index.values,
                                feature_names=features, 
                                out_file=None,
                                filled=True, 
                                special_characters=True, 
                                rounded=True 
                               )
tree = graphviz.Source(tree_data)
tree.format = 'png'
tree.render("DecisionTree")

'DecisionTree.png'

<img src="DecisionTree.png">


## Regresión logística

In [17]:
trainX.shape, trainY.shape, testX.shape, testY.shape, valX.shape, valY.shape

((605, 11), (605,), (179, 11), (179,), (107, 11), (107,))

In [18]:
trainXPlusB = np.column_stack((np.ones(trainX.shape[0]), trainX))
valXPlusB = np.column_stack((np.ones(valX.shape[0]), valX))

print(np.shape(trainXPlusB),np.shape(valXPlusB))

fil, col = trainXPlusB.shape

(605, 12) (107, 12)


### Definición del grafo

In [19]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():   
    
    #Placeholder
    lr = tf.placeholder(tf.float32,name="lr")
    x  = tf.placeholder(tf.float32,shape=(None,col),name="x")
    y  = tf.placeholder(tf.float32, name = "y")
    pLambda = tf.placeholder(tf.float32, name = "pLambda")

    #Variables
    with tf.name_scope("parametros"):
        w = tf.Variable(tf.truncated_normal(shape = [col, 1]), name = "w")

    #Calculando logits y estimaciones
    with tf.name_scope("Logits"):
        logits = tf.matmul(x, w, name = "logits") 
        yHat = tf.nn.sigmoid(logits, name = "YHat")
        
    #Regularizando
    with tf.name_scope("RegularizacionW"):
        w_reg = tf.divide(tf.multiply(tf.multiply(tf.constant(0.5), pLambda),tf.reduce_sum(tf.square(w))), tf.cast(fil, tf.float32), name = "w_reg")

    #Definiendo función de costo    
    with tf.name_scope("costoSinRegularizacion"):        
        costoSinRegularizacion = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = logits), name = "costoSinRegularizacion")
    with tf.name_scope("costo"):
        costo = tf.add(costoSinRegularizacion, w_reg, name="Costo")    

    # Optimizador de gradiente 
#   with tf.name_scope("OptimizadorGradiente"):
#   gradienteOptimizado = tf.train.GradientDescentOptimizer(lr).minimize(costo) 
        
    #Autodiferenciación para calcular gradiente
    with tf.name_scope("Gradiente"):
        grad = tf.gradients(costo,w)
        
    #Descenso de gradiente    
    with tf.name_scope("actualizado_parametros"):
        new_params = tf.assign(w, w - lr*grad[0] )

    #Calculando métricas   
    with tf.name_scope("Metricas"):
        M_costo = tf.summary.scalar(name='costo', tensor = costo)
        metricas = tf.summary.merge_all()

    init = tf.global_variables_initializer()


### Función para entrenar el modelo

In [20]:
def entrenar_modelo_regresion_logistica(x_vector,y_vector,epochs,imprimir_error_cada,learning_rate,batch_size,tipoRegularizacion,p_lambda):
    # Iniciando grafo
    with tf.Session(graph = g) as session:
        # Inicializamos las variables en la sesión
        session.run(init) 
        # Manejo de los logs en tensorboard
        writer = tf.summary.FileWriter('./tensorBoardLogs/'+ datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
                                       + '_GadientDescent_lr=' + str(learning_rate)
                                       + '_epochs=' + str(epochs)
                                       + '_tipoRegularizacion=' + str(tipoRegularizacion), session.graph)


        tamaño_muestra=x_vector.shape[0] 
        total_iteraciones = int(tamaño_muestra/batch_size)

        count=0    
        for epoch in range(epochs):
            for i in range(total_iteraciones):
                count=count+1
                muestra_inicio = i*batch_size
                muestra_fin = muestra_inicio + batch_size

                x_mb = np.array(x_vector[muestra_inicio:muestra_fin])
                y_mb = np.array(y_vector[muestra_inicio:muestra_fin])
                feed_dict = {x:x_mb, y:y_mb, lr:learning_rate, pLambda:p_lambda}

#                 _, metrics = session.run([gradienteOptimizado, metricas], feed_dict = feed_dict)
                _, metrics = session.run([new_params, metricas], feed_dict = feed_dict)
                writer.add_summary(metrics, count)

                if ((i + 1) % imprimir_error_cada == 0): 
                    cost = session.run(costo, feed_dict = feed_dict)
                    print("Epoch = %d,\tIteración = %d,\tCosto: %0.4f" % (epoch+1, i+1, cost))
        writer.close()
        params = session.run(w)

    return params

In [21]:
def predecir_modelo_regresion_logistica(x, w):
    logits = np.matmul(x, w)
    yHat = ((1 / (1 + np.exp(-logits))) > 0.5)*1.0
    return yHat    

In [22]:
def metricas_modelo_regresion_logistica(lambda_,lr_,xVal, yVal, w):
    yHat = predecir_modelo_regresion_logistica(xVal, w)
    precision = metrics.precision_score(yVal, yHat)
    accuracy = metrics.accuracy_score(yVal, yHat)
    recall = metrics.recall_score(yVal, yHat)
    f1_score = metrics.f1_score(yVal, yHat)
 
    resultados = { 'lambda':[lambda_], 'lr':[lr_],'precision':[precision],'accuracy':[accuracy],
                   'recall':[recall],'f1_score':[f1_score]
                 }
    df_resultados = pd.DataFrame(resultados)
    bitacora('bitacoraRegresionLogistica.csv',df_resultados)
    return df_resultados


In [23]:
lista_lr = np.arange(0.001,0.009,0.001)
lista_lambda = np.arange(1,6,1)
resultado_modelos_RegLogistica= pd.DataFrame()
modelos_RegLogistica = list()

for lr_ in lista_lr:
    for lambda_ in lista_lambda:
        model_RegLosgistica = entrenar_modelo_regresion_logistica(trainXPlusB,trainY,10,50,lr_,30,"L2",lambda_)
        modelos_RegLogistica.append(model_RegLosgistica)
        resultados = metricas_modelo_regresion_logistica(lambda_,lr_,valXPlusB, valY, model_RegLosgistica)
        resultado_modelos_RegLogistica = pd.concat([resultado_modelos_RegLogistica,resultados],axis=0,ignore_index=True)
    
    

index_best_model = resultado_modelos_RegLogistica['accuracy'].idxmax() 
np.savez(file = "modeloLogisticRegression.npz", w = modelos_RegLogistica[index_best_model])
print(resultado_modelos_RegLogistica.loc[index_best_model])


lambda       3.000000
lr           0.003000
precision    0.750000
accuracy     0.682243
recall       0.391304
f1_score     0.514286
Name: 12, dtype: float64


### Grafo, costo y exactitud en Tensorboard

<img src="TensorBoard.png">

<img src="Metricas.png">

## Naive Bayes con pandas

In [24]:
def naive_bayes_pandas(trainXv, trainYv, predictColumn, predictValue):
    summ = pd.DataFrame(columns=['Feature', 'Element', 'Prob'])
    sCondicional = pd.DataFrame(columns=['Feature', 'YItem', 'Element', 'Prob'])
    y_total_rows = trainYv.shape[0]
    objective_features = trainYv[predictColumn].unique().tolist()
    numUpper = 1
    numDown = 1
    for feature in objective_features:
        probValue = trainYv[trainYv == feature].count() / y_total_rows
        summ = summ.append({'Feature': predictColumn, 'Element': feature, 'Prob': probValue}, ignore_index=True)
    x_total_rows = trainXv.shape[0]
    x_colnames = trainXv.columns
    for column in x_colnames:
        FeaturesValues = trainXv[column].unique().tolist()
        for feature in FeaturesValues:
            features = trainXv[column]
            probValue = features[features == feature].count() / x_total_rows
            summ = summ.append({'Feature': column, 'Element': feature, 'Prob': probValue}, ignore_index=True)
            for y_feature in objective_features:
                cFeatures = pd.DataFrame(columns=['YColumn', 'XColumn'])
                cFeatures['XColumn'] = features
                cFeatures['YColumn'] = trainYv
                cFeatures = cFeatures[cFeatures['YColumn']==y_feature]
                predFeatures_count = len(cFeatures)
                cFeatures = cFeatures[cFeatures['XColumn']==feature]
                cFeatures_count = len(cFeatures)
                sCondicional = sCondicional.append({'Feature': column, 'YItem': y_feature, 'Element': feature, 'Prob': cFeatures_count/predFeatures_count}, ignore_index=True)
    trainYv_count = len(trainYv)
    predictValue_count = len(trainYv[trainYv==predictValue])
    predictprobValue = predictValue_count / trainYv_count 
    x_colnames = trainXv.columns
    resultado = pd.DataFrame(columns=['Feature', 'Element', 'Prob'])
    for column in x_colnames:
        FeaturesValues = trainXv[column].unique().tolist()
        for feature in FeaturesValues:
            ProbConditional = sCondicional[(sCondicional['YItem']==predictValue) & (sCondicional['Feature']==column) & (sCondicional['Element']==feature)]            
            probX = summ[(summ['Feature']==column) & (summ['Element']==feature)]            
            numUpper = ProbConditional['Prob'].iloc[0] * predictprobValue
            numDown = probX['Prob'].iloc[0]
            resultado = resultado.append({'Feature': column, 'Element': feature, 'Prob': numUpper/numDown}, ignore_index=True)
    return resultado

## Naive Bayes con SKLearn

In [25]:
def train_NaiveBayes(xTrain, yTrain, xVal, yVal):    
    model = GaussianNB()
    modelNaiveBayes = model.fit(xTrain,yTrain)
    yHat = modelNaiveBayes.predict(xVal)    
    precision = metrics.precision_score(yVal, yHat)
    accuracy = metrics.accuracy_score(yVal, yHat)
    recall = metrics.recall_score(yVal, yHat)
    f1_score = metrics.f1_score(yVal, yHat)
    score = modelNaiveBayes.score(xVal, yVal)  
    resultados = { 'precision':[precision],'accuracy':[accuracy],
                   'recall':[recall],'f1_score':[f1_score],
                   'score':[score]
                 }
    df_resultados = pd.DataFrame(resultados)
    bitacora('bitacoraNaiveBayes.csv',df_resultados)
    return df_resultados,modelNaiveBayes

In [26]:
resultado_modelo_NaiveBayes, model_NaiveBayes = train_NaiveBayes(trainX, trainY, valX, valY)
print(resultado_modelo_NaiveBayes)
joblib.dump(model_NaiveBayes, 'modeloNaiveBayes.pkl') 

   precision  accuracy    recall  f1_score    score
0   0.727273   0.71028  0.521739  0.607595  0.71028


['modeloNaiveBayes.pkl']

## Ensemble

In [27]:
def Ensemble(X):
    valXPlusB = np.column_stack((np.ones(X.shape[0]), X))
    svmModel = joblib.load("modeloSVM.pkl")
    desicionTreeModel = joblib.load("modeloDecisionTree.pkl")
    logisticalRegression = np.load("modeloLogisticRegression.npz")
    w = logisticalRegression['w']
    naiveBayesModel = joblib.load("modeloNaiveBayes.pkl")
    yHat_SVM = svmModel.predict(X)
    yHat_desicionTree = desicionTreeModel.predict(X)
    yHat_logisticalRegression = predecir_modelo_regresion_logistica(valXPlusB, w)
    yHat_naiveBayes = naiveBayesModel.predict(X)
    yHats = np.column_stack((yHat_SVM,yHat_desicionTree,yHat_logisticalRegression,yHat_naiveBayes))
    yHat, _ = stats.mode(yHats, axis = 1)
    return yHat, yHats


## Ensemble datos de evaluación

In [28]:
yHat, yHats = Ensemble(valX)
EnsembleResults = pd.DataFrame(np.column_stack((yHats, yHat)), 
             columns=["svmModel", "desicionTreeModel Tree","logisticalRegression Logistica","naiveBayesModel", "Survived"])
EnsembleResults

,svmModel,desicionTreeModel Tree,logisticalRegression Logistica,naiveBayesModel,Survived
0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
102,0.0,1.0,0.0,0.0,0.0
103,0.0,1.0,0.0,0.0,0.0
104,0.0,1.0,0.0,0.0,0.0
105,0.0,0.0,0.0,0.0,0.0


## Ensemble datos de prueba final

In [29]:
scaler = StandardScaler()
testX = scaler.fit_transform(testX)

yHatTest, yHatsTest = Ensemble(testX)
EnsembleResultsTest = pd.DataFrame(np.column_stack((yHatsTest, yHatTest)), 
             columns=["svmModel", "desicionTreeModel Tree","logisticalRegression Logistica","naiveBayesModel", "Survived"])
EnsembleResultsTest


,svmModel,desicionTreeModel Tree,logisticalRegression Logistica,naiveBayesModel,Survived
0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...
174,0.0,0.0,1.0,1.0,0.0
175,0.0,0.0,0.0,1.0,0.0
176,0.0,1.0,0.0,1.0,0.0
177,0.0,0.0,0.0,0.0,0.0


In [30]:
#Accurrancy
mt.accuracy_score(testY, yHatTest)

0.8100558659217877

## Conclusiones y recomendaciones

Una de las recomendaciones que forman parte de las primeras tareas al aboradar problemas en machine learning es realizar un anális exploratorio de datos, esto permite comprender de mejor manera la cantidad, forma y tipo de datos, a partir de los cuales tendremos que aplicar distintas transformaciones a los mismos para poder iniciar con la definición y entrenamiento de modelos de machine learning.

En mi opinion, entrenar y validar modelos de machine learning requiere realizar varios experimentos variando los distintos hiperparametros de cada modelo, esto permite ir mejorando poco a poco la calidad del modelo buscando, es una tarea que requiere de tiempo y esfuerzo considerable pero en el camino se va ganando experiencia y se logra una mejor comprensión de cómo cada uno de los parámetros afectan el resultado final. 

Una de las mayores dificultades encontradas en el proyecto es la investigación, interpretación y aplicación de las distintas funciones para clasificación binaria, ya que entre la teoría y la práctica a veces se vuelve un poco confuso la interpretación de las mismas. Lectura recomendada: https://stackoverflow.com/questions/46291253/what-is-the-difference-between-a-sigmoid-followed-by-the-cross-entropy-and-sigmo

Utilizar un conjunto de modelos de machine learning en donde cada uno produce una predicción diferente y luego combinar las mismas para obtener una única predicción (Ensemble) permite tener un modelo robusto en donde por la naturaleza diferente de cada uno de los modelos que se combinan, los errores tienden a compensarse. Cuando se desea predecerir un dato nuevo se obtiene la predicción de cada modelo y la predicción resultante será la que la mayoría de los modelos presentaron.



#### Bootstraping

Consiste en obtener muestras de datos aleatorias de todo el dataset, en el proyecto no fue aplicado pero puede utilizarse la función **numpy.random.sample (utilizando el parámetro -> replace = True)** 

#### k-fold cross validation

Es una técnica que consiste en dividir el dataset de forma aleatoria en **k** grupos de aproximadamente el mismo tamaño. **k-1 grupos** se emplean para entrenar el modelo y uno de los grupos se emplea como test, este proceso se repite **k** veces utilizando un grupo distinto como test en cada iteración. El proceso genera k estimaciones del error cuyo promedio se emplea como estimación final.

